In [ ]:
import random

import pyomo.environ as penv
import pyomo.opt as popt

We formulate a stochastic variant of the maximum flow model to demonstrate how to implement constraints on the distributions of loss for various metrics. In each scenario, the flow across an edge is limited to the minimum of a deterministic upper bound and the sum of the uncertain reduced capacity with the first-stage capacity reinforcement.

The objective is to minimize the sum of all first-stage capacity reinforcement decisions such that the loss distribution constraints are satisfied.

In [ ]:
nodes = [0, 1, 2, 3, 4, 5]
edges = {
    (0, 1): 16,
    (0, 2): 13,
    (1, 3): 12,
    (1, 2): 10,
    (2, 1):  4,
    (2, 4): 14,
    (3, 2):  9,
    (4, 3):  7,
    (4, 5):  4,
    (3, 5): 20,
    (5, 0): 99,
}
source = 0
sink = 5

In [ ]:
num_scenarios = 100
seed = 0
random.seed(seed)
xi = {(n, m, scenario): edges[source, sink]
                        if (n, m) == (source, sink) else
                        random.randint(0, edges[n, m])
      for (n, m) in edges
      for scenario in range(num_scenarios)}

# Core Model

$$
\begin{align}
\min & \sum_{(n,m) \in E} x_{n,m} \\
\text{s.t.} & \sum_{m \in \delta^+(n)} y_{m,n}^\omega = \sum_{m \in \delta^-(n)} y_{n,m}^\omega, \quad \forall n \in N, \forall \omega \in \Omega \\
& y_{n,m}^\omega \le f_{n,m}, \quad \forall (n,m) \in E, \forall \omega \in \Omega \\
& y_{n,m}^\omega \le \xi_{n,m}^\omega + x_{n,m}, \quad \forall (n,m) \in E, \forall \omega \in \Omega
\end{align}
$$

In [ ]:
model = penv.ConcreteModel()
model.nodes = penv.Set(initialize=nodes)
model.edges = penv.Set(initialize=list(edges))
model.scenarios = penv.Set(initialize=range(num_scenarios))
model.s = source
model.t = sink
model.flow_capacity = penv.Param(model.edges, initialize=edges)
model.xi = penv.Param(model.edges, model.scenarios, initialize=xi)

# x are first-stage branch hardening decisions
model.x = penv.Var(model.edges, domain=penv.NonNegativeReals)
# y are second-stage branch flow decisions
model.y = penv.Var(model.edges, model.scenarios, domain=penv.NonNegativeReals)

def con_flow_balance(model, node, scenario):
    in_edges = [(n, m) for (n, m) in model.edges if m == node]
    out_edges = [(n, m) for (n, m) in model.edges if n == node]
    return sum(model.y[edge, scenario] for edge in in_edges) == sum(model.y[edge, scenario] for edge in out_edges)

def con_max_flow_capacity(model, n, m, scenario):
    return model.y[n, m, scenario] <= model.flow_capacity[n, m]

def con_eff_flow_capacity(model, n, m, scenario):
    return model.y[n, m, scenario] <= model.xi[n, m, scenario] + model.x[n, m]

def obj_min_budget(model):
    return sum(model.x[edge] for edge in model.edges)

model.con_flow_balance = penv.Constraint(model.nodes, model.scenarios, rule=con_flow_balance)
model.con_max_flow_capacity = penv.Constraint(model.edges, model.scenarios, rule=con_max_flow_capacity)
model.con_eff_flow_capacity = penv.Constraint(model.edges, model.scenarios, rule=con_eff_flow_capacity)
model.obj_min_budget = penv.Objective(sense=penv.minimize, rule=obj_min_budget)

# Type I Service-Level Constraints

$$
\begin{align}
& \text{Pr}(\boldsymbol{y}_{t,s} \ge b_i) \ge \alpha_i, \quad i = 1, \ldots, N_\text{Type-I} \\
& \Updownarrow \\
& \mathcal{I}_i^\omega \le \frac{y_{t,s}^\omega}{\bar{y}_{t,s}}, \quad \forall \omega \in \Omega, i = 1, \ldots, N_\text{Type-I}, \\
& \sum_{\omega \in \Omega} \pi^\omega \mathcal{I}_i^\omega \ge \alpha_i, \quad i = 1, \ldots, N_\text{Type-I}
\end{align}
$$


In [ ]:
def con_indicator(model, scenario, type1_con):
    return model.indicator[scenario, type1_con] <= model.y[model.t, model.s, scenario] / perf_thresholds[type1_con]

def con_type1_service_level(model, type1_con):
    return sum(model.indicator[scenario, type1_con] for scenario in model.scenarios) / num_scenarios >= alpha_thresholds[type1_con]

perf_thresholds = [20, 18, 15]
alpha_thresholds = [0.50, 0.75, 1.00]
num_type1_constraints = len(perf_thresholds)

model.type1_con_set = penv.Set(initialize=range(num_type1_constraints))
model.indicator = penv.Var(model.scenarios, model.type1_con_set, domain=penv.Binary)
model.con_indicator = penv.Constraint(model.scenarios, model.type1_con_set, rule=con_indicator)
model.con_type1_service_level = penv.Constraint(model.type1_con_set, rule=con_type1_service_level)

# Type II Service-Level Constraints

$$
\begin{align}
& \frac{\mathbb{E}[\boldsymbol{y}_{t,s}]}{\bar{y}_{t,s}} \ge \beta \\
& \Updownarrow \\
& \frac{\displaystyle\sum_{\omega \in \Omega} \pi^\omega y_{t,s}^\omega}{\bar{y}_{t,s}} \ge \beta
\end{align}
$$

In [ ]:
def con_type2_service_level(model):
    return sum(model.y[model.t, model.s, scenario] for scenario in model.scenarios) / num_scenarios >= perf_target * beta_threshold

perf_target = 23
beta_threshold = 0.75

model.con_type2_service_level = penv.Constraint(rule=con_type2_service_level)

# CVaR Constraints

$$
\begin{align}
& \text{CVaR}^{\epsilon_i} [\boldsymbol{y}_{t,s}] \le \zeta_i, \quad i = 1, \ldots, N_\text{CVaR} \\
& \Updownarrow \\
& \beta_i + \sum_{\omega \in \Omega} \pi^\omega \nu_i^\omega \le \zeta_i, \quad i = 1, \ldots, N_\text{CVaR}, \\
& \nu_i^\omega \ge \bar{y}_{t,s} - y_{t,s}^\omega - \beta_i, \quad \forall \omega \in \Omega, i = 1, \ldots, N_\text{CVaR}, \\
& \nu_i^\omega \ge 0, \quad \forall \omega \in \Omega, i = 1, \ldots, N_\text{CVaR}
\end{align}
$$

In [ ]:
def con_cvar_main(model, cvar_con):
    return model.cvar_beta[cvar_con] + sum(model.cvar_nu[scenario, cvar_con] for scenario in model.scenarios) / (num_scenarios * cvar_epsilons[cvar_con]) <= cvar_thresholds[cvar_con]

def con_cvar_aux(model, scenario, cvar_con):
    loss = perf_target - model.y[model.t, model.s, scenario]
    return model.cvar_nu[scenario, cvar_con] >= loss - model.cvar_beta[cvar_con]

perf_target = 23
cvar_thresholds = [4, 7.5]
cvar_epsilons = [0.50, 0.10]
num_cvar_constraints = len(cvar_thresholds)

model.cvar_con_set = penv.Set(initialize=range(num_cvar_constraints))
model.cvar_beta = penv.Var(model.cvar_con_set)
model.cvar_nu = penv.Var(model.scenarios, model.cvar_con_set, domain=penv.NonNegativeReals)
model.con_cvar_main = penv.Constraint(model.cvar_con_set, rule=con_cvar_main)
model.con_cvar_aux = penv.Constraint(model.scenarios, model.cvar_con_set, rule=con_cvar_aux)

In [ ]:
solver = popt.SolverFactory('gurobi')
result = solver.solve(model)

for key in model.x:
    model.x[key].fix(model.x[key].value)

model.obj_min_budget.deactivate()

def obj_max_flow(model):
    return sum(model.y[model.t, model.s, scenario] for scenario in model.scenarios) / num_scenarios
model.obj_max_flow = penv.Objective(sense=penv.maximize, rule=obj_max_flow)

result = solver.solve(model)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

probabilities = np.linspace(0.01, 1.00, 100)
losses = np.array([23 - model.y[model.t, model.s, scenario].value for scenario in model.scenarios])

plt.plot(probabilities, np.sort(losses), 'k-')

# Type I
for i in range(num_type1_constraints):
    plt.plot([alpha_thresholds[i]] * 2, [0, perf_target - perf_thresholds[i]], '-', color='green')
    plt.plot([0, alpha_thresholds[i]], [perf_target - perf_thresholds[i]] * 2, '-', color='green')
    idx = int(np.floor(alpha_thresholds[i] * num_scenarios) - 1)
    plt.plot([0, alpha_thresholds[i]], [np.sort(losses)[idx]] * 2, color='lime')

# Type II
plt.plot([0, 1], [perf_target * (1 - beta_threshold)] * 2, '-', color='red')
plt.plot([0, 1], [np.mean(losses)] * 2, '--', color='orange')

# CVaR
for i in range(num_cvar_constraints):
    plt.plot([1 - cvar_epsilons[i], 1], [cvar_thresholds[i]] * 2, '-', color='blue')
    idx = int(np.floor((1 - cvar_epsilons[i]) * num_scenarios) - 1)
    plt.plot([1 - cvar_epsilons[i], 1], [np.sort(losses)[idx:].mean()] * 2, '--', color='cyan')

plt.show()